In [ ]:
import numpy as np
import os
import visvis as vv
import matplotlib.pyplot as plt
import pyproj
import pandas as pd
import matplotlib

import geovis_notebook_version

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass

# Load voxel data

In [ ]:
dir_year = '2018'
dir_month = '11'
dir_day = '15'
dir_experiment_num = '01'
sub_dir = '{}_{}_{}/{}'.format(dir_month, dir_day, dir_year, dir_experiment_num)
experiment_str = '{}-{}-{}-{}'.format(dir_month, dir_day, dir_year, dir_experiment_num)
dir_voxels = '/Volumes/david_hd/obsidian/output/experiments/{}/voxels/'.format(sub_dir)
dir_plot_output = '/Users/davidkohn/Desktop'

In [ ]:
fname_voxels_list = [
     dir_voxels + f for f in os.listdir(dir_voxels) 
    if os.path.isfile(os.path.join(dir_voxels, f))
    and 'voxel' in f
]
print(fname_voxels_list)

In [ ]:
# add voxels in
for idx, fn in enumerate(fname_voxels_list):
    print(idx)
    if idx == 0:
        view = geovis_notebook_version.MasonView(fn)
    else:
        view.add_samples(geovis_notebook_version.MasonView(fn))

In [ ]:
# centre point
C_lon = 116.1
C_lat = -24.85

# mason voxels per axis
V_x = 32
V_y = 32
V_z = 32

# we've specified a 10km x 10km area in the experiments
# length is in metres
length_of_square = 10000

# pointA is the start of the line, pointB is the end of the line
pointA_ne = (409000, 7244000)
pointB_ne = (414000, 7257000)

In [ ]:
# generate coord_array_ne that represents the mason voxelised posterior grid in (northings, eastings)
gda94 = pyproj.Proj(init='epsg:4283')
mgaz50 = pyproj.Proj(init='epsg:28350')

# centre point
C_lonlat = (C_lon, C_lat)
print('C (longitude, latitude): {}'.format(C_lonlat))
C_ne = pyproj.transform(gda94, mgaz50, *C_lonlat)
print('C (northings, eastings): {}'.format(C_ne))

# define the location of the upper left hand corner edge point
E_ne = tuple(np.array(C_ne) + [-1 * 0.5 * length_of_square, -1 * 0.5 * length_of_square])
print('E (northings, eastings): {}'.format(E_ne))

# all the key points
middle = np.array(C_ne)
lower_left = middle + [-5000, -5000]
top_left = middle + [-5000, 5000]
top_right = middle + [5000, 5000]
lower_right = middle + [5000, -5000]
point_arr = np.array([middle, lower_left, top_left, top_right, lower_right])

# mason voxels per axis
#coord_array_ne = np.zeros((V_x, V_y), dtype=(float, 2))
y_list = np.linspace(lower_left[1], top_left[1], 32)
x_list = np.linspace(lower_left[0], lower_right[0], 32)
grid_x, grid_y = np.meshgrid(x_list, y_list)

ab_arr = np.array([pointA_ne, pointB_ne])

In [ ]:
#plt.scatter(point_arr[:,0], point_arr[:,1], c='red')
fig = plt.figure(figsize=(10,10))
ax = plt.gca()
rect = matplotlib.patches.Rectangle(lower_left, length_of_square, length_of_square,edgecolor='r',facecolor='none')
plt.plot(ab_arr[:,0], ab_arr[:,1], c='blue', marker='o')
ax.add_patch(rect)
ax.set_xlim([400000, 420000])

In [ ]:
f = plt.figure(figsize=(20,20))
tick_fontsize = 20
ax = plt.gca()
plt.scatter(grid_x, grid_y)
plt.plot(ab_arr[:,0], ab_arr[:,1], c='red', marker='o')
_ = ax.set_xticks(x_list)
_ = ax.set_xticklabels(range(len(x_list)))
_ = ax.set_yticks(y_list)
_ = ax.set_yticklabels(range(len(y_list)))
_ = plt.yticks(fontsize = tick_fontsize)
_ = plt.xticks(fontsize = tick_fontsize)
ax.grid()
plt.tight_layout()
fpath = os.path.join(dir_plot_output, 'chord.eps')
plt.savefig(fpath)

In [ ]:
# calculate slope between pointA and pointB
northings_diff = (pointB_ne[0] - pointA_ne[0])
eastings_diff = (pointB_ne[1] - pointA_ne[1])
m = northings_diff / eastings_diff
# create array all points along the cross section
new_point_array = []
#modifier = 450
modifier = 0
step = int(np.floor(np.diff(x_list)[0])) + modifier
for idx, d in enumerate(range(pointA_ne[1], pointB_ne[1] + step, step)):
    new_e = (pointA_ne[1] + (step * idx) )
    new_n = m * (step * idx) + pointA_ne[0]
    new_point = (new_n, new_e)
    new_point_array.append(new_point)
new_point_array = np.array(new_point_array)

In [ ]:
x_plot_list = []
y_plot_list = []

x_ne_list = []
y_ne_list = []

for coord_ne_i in new_point_array:
    x_i = coord_ne_i[0]
    y_i = coord_ne_i[1]
    x_true = [x_i > x for x in x_list]
    y_true = [y_i > y for y in y_list]
    #print('point', x_i, y_i)
    if np.any(x_true) and (np.any(y_true)):
        x_max = np.max(np.where(x_true))
        y_max = np.max(np.where(y_true))
        """
        print('cell', x_list[x_max], y_list[y_max])
        print('indices', x_max, y_max)
        print()
        """
        x_plot_list.append(x_max)
        y_plot_list.append(y_max)
        x_ne_list.append(x_i)
        y_ne_list.append(y_i)

In [ ]:
df_plot = pd.DataFrame([x_plot_list, y_plot_list]).T.drop_duplicates()
df_plot_ne = pd.DataFrame([x_ne_list, y_ne_list]).T.drop_duplicates()

In [ ]:
grid_x.shape

In [ ]:
occupancy_array = np.zeros(grid_x.shape)
for x,y in df_plot.values:
    occupancy_array[x,y] = 1

In [ ]:
# plot the voxel occupancy map
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 5
plot_height = 5
fontsize = 20
tick_fontsize = 20

x_label = 'Eastings'
y_label = 'Northings'

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
ax.set_adjustable('box-forced')
plt.yticks(fontsize = tick_fontsize)
plt.xticks(fontsize = tick_fontsize)
plt.xlabel(x_label, fontsize = fontsize)
plt.ylabel(y_label, fontsize = fontsize)

f = ax.imshow(occupancy_array.T, cmap=plt.cm.binary, aspect='auto', origin = 'lower')
ax.set_xticks(range(0, 33, 4))
ax.set_yticks(range(0, 33, 4))
#ax.grid(which='major', color='k', linestyle='-', linewidth=1)
plt.tight_layout()
fpath = os.path.join(dir_plot_output, 'cross-section-line.eps')
plt.savefig(fpath)

In [ ]:
# select a layer to plot
layer_idx = 1
layer_names = ['layer 0', 'layer 1', 'layer 2']
layer_voxels = view.meanlayer(layer_idx)
print('layer_voxels.shape: {}'.format(layer_voxels.shape))

# get z-values for each surface voxel selected
print(layer_voxels.shape)
# x is third dim
# y is second dim
# z if first dim
x_indices = df_plot[0]
y_indices = df_plot[1]
depth_voxels = []
for x, y in zip(x_indices, y_indices):
    z_i = layer_voxels[:, y, x]
    depth_voxels.append(z_i)
depth_voxels = np.array(depth_voxels).T

In [ ]:
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 5
plot_height = 5
fontsize = 20
tick_fontsize = 29

x_label = 'Distance Along Chord (km)'
y_label = 'Depth (km)'

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
plt.yticks(fontsize = tick_fontsize)
plt.xticks(fontsize = tick_fontsize)
plt.xlabel(x_label, fontsize = fontsize)
plt.ylabel(y_label, fontsize = fontsize)


plt.contourf(depth_voxels, origin = 'upper')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=20)
v = np.linspace(0, 1, 5, endpoint=True)
cbar.set_ticks(v)
cbar.set_clim([0, 1])
cbar.set_label('Probability', fontsize=fontsize)

#ax.set_ylim([0.5, 31.5])
ax.set_ylim([0.5, 4.5])
ytl = ['{:.0f}'.format(ytl_i) for ytl_i in ax.get_yticks()]
ax.set_yticks(np.arange(31.5, 0.5, -5))

#ax.invert_yaxis()
#ax.set_xticks(list(range(0, 28, 4)) + [28])
ax.set_yticklabels(ytl)

plt.tight_layout()

fpath = os.path.join(dir_plot_output, 'cross-section-{}.eps'.format(layer_names[layer_idx]))
plt.savefig(fpath)

In [ ]:
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 5
plot_height = 5
fontsize = 20
tick_fontsize = 29

x_label = 'Distance Along Chord (km)'
y_label = 'Depth (km)'

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
plt.yticks(fontsize = tick_fontsize)
plt.xticks(fontsize = tick_fontsize)
plt.xlabel(x_label, fontsize = fontsize)
plt.ylabel(y_label, fontsize = fontsize)


plt.contourf(depth_voxels, origin = 'upper')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=20)
v = np.linspace(0, 1, 5, endpoint=True)
cbar.set_ticks(v)
cbar.set_clim([0, 1])
cbar.set_label('Probability', fontsize=fontsize)

y_max = 31.5
ax.set_ylim([0.5, y_max])
ytl = ['{:.0f}'.format(ytl_i) for ytl_i in ax.get_yticks()]
ax.set_yticks(np.arange(31.5, 0.5, -5))
#ax.invert_yaxis()
ax.set_ylim([y_max - 5, y_max])

ax.set_yticklabels(ytl)

plt.tight_layout()

fpath = os.path.join(dir_plot_output, 'cross-section-small-{}.eps'.format(layer_names[layer_idx]))
plt.savefig(fpath)

In [ ]:
f = plt.figure(figsize=(20,20))
ax = plt.gca()
plt.scatter(grid_x, grid_y)
plt.plot(ab_arr[:,0], ab_arr[:,1], c='red', marker='o')
plt.scatter(df_plot_ne[0], df_plot_ne[1], c='green', marker='*',s=200)
#plt.imshow(df_plot_ne.values)
_ = ax.set_xticks(x_list)
_ = ax.set_xticklabels(range(len(x_list)))
_ = ax.set_yticks(y_list)
_ = ax.set_yticklabels(range(len(y_list)))
_ = plt.yticks(fontsize = tick_fontsize)
_ = plt.xticks(fontsize = tick_fontsize)
ax.grid()
plt.tight_layout()
fpath = os.path.join(dir_plot_output, 'chord-marked.eps')
plt.savefig(fpath)